In [15]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi,cos,sin
from scipy.linalg import logm

In [16]:
#将从示教器读取的数据转化为旋转矩阵和平移矩阵
def data_transform(M):
    T=np.array(M[:3]).reshape(-1,1)
    rota=M[3]*pi/180
    rotb=M[4]*pi/180
    rotc=M[5]*pi/180
    Rx=np.array([[1,0,0],[0,cos(rota),-sin(rota)],[0,sin(rota),cos(rota)]])
    Ry=np.array([[cos(rotb),0,sin(rotb)],[0,1,0],[-sin(rotb),0,cos(rotb)]])
    Rz=np.array([[cos(rotc),-sin(rotc),0],[sin(rotc),cos(rotc),0],[0,0,1]])
    R=np.dot(Rz,np.dot(Ry,Rx))
    h_stack=np.hstack((R,T))
    one=np.array([0,0,0,1])
    TR=np.vstack((h_stack,one))
    return TR

In [17]:
def skew(x):
    sk=np.array([[0,-x[2,0],x[1,0]],
                 [x[2,0],0,-x[0,0]],
                 [-x[1,0],x[0,0],0]
                 ])
    return sk

In [18]:
def calculate_fixed_point(fix_point,R,position):
    k1=(fix_point[3]-fix_point[1])/(fix_point[2]-fix_point[0])#k1=(y2-y1)/(x2-x1)
    k2=(fix_point[7]-fix_point[5])/(fix_point[6]-fix_point[4])#k2=(y4-y3)/(x4-x3)
    k11=-1/k1#垂直平分线1的斜率
    k22=-1/k2#垂直平分线2的斜率
    x01=(fix_point[0]+fix_point[2])/2#x01=(x1+x2)/2——中点1的横坐标
    y01=(fix_point[1]+fix_point[3])/2#y01=(y1+y2)/2——中点1的纵坐标
    x02=(fix_point[4]+fix_point[6])/2#x02=(x3+x4)/2——中点2的横坐标
    y02=(fix_point[5]+fix_point[7])/2#y02=(y3+y4)/2——中点2的纵坐标
    x=((k11*x01-y01)-(k22*x02-y02))/(k11-k22)
    y=k11*(x-x01)+y01#(x,y)为圆心O'的坐标
    r2=(x-fix_point[0])**2+(y-fix_point[1])**2
    #得到圆心的x,y坐标后还需要求解z坐标
    distance=(R**2-r2)**0.5#z方向的垂直距离
    if position==1:
        z=-distance
    else:
        z=distance
    circle_point=np.array([x,y,z]).reshape(-1,1)
    I=np.eye(3)
    h_stack=np.hstack((I,circle_point))
    one=np.array([0,0,0,1])
    TC=np.vstack((h_stack,one))
    return TC

In [19]:
def axis_transformtion(x):
    result=[]
    for i in [0,2,4,6]:
        x0=x[i]*cos(x[i+1]/180*pi)
        y0=x[i]*sin(x[i+1]/180*pi)
        result.append(x0)
        result.append(y0)
    return result

In [20]:
#通过tasi算法计算手眼标定矩阵
#函数说明：计算AX=XB解的最小方差
#算法作者:Lenz Shah
def tasi(A,B):
    n=int(A.shape[1]/4)#获取方程的个数
    S=np.zeros((3*n,3))#用于存储计算得到的旋转矩阵
    v=np.zeros((3*n,1))#用于存储计算得到的平移矩阵
    #计算最佳的旋转矩阵R
    for i in range(n):
        A1=logm(A[0:3,4*i:4*i+3])
        B1=logm(B[0:3,4*i:4*i+3])
        a=np.array([A1[2,1],A1[0,2],A1[1,0]]).reshape(-1,1)
        a=a/np.linalg.norm(a)
        b=np.array([B1[2,1],B1[0,2],B1[1,0]]).reshape(-1,1)
        b=b/np.linalg.norm(b)
        S[3*i:3*(i+1),:]=skew(a+b)
        v[3*i:3*(i+1),:]=a-b
    x,residuals,rank,s=np.linalg.lstsq(S,v,rcond=None)#求解AX=B线性方程组的解
    theta=2*np.arctan(np.linalg.norm(x))
    I=np.eye(3)
    xxt=np.outer(x,x)#计算外积矩阵
    R_T=I*np.cos(theta)+np.sin(theta)*skew(x)+(1-cos(theta))*xxt
    R=R_T.T
    #计算最佳平移矩阵T
    C=np.zeros((3*n,3))
    d=np.zeros((3*n,1))
    for i in range(n):
        C[3*i:3*(i+1),:]=I-A[0:3,4*i:4*i+3]
        d[3*i:3*(i+1),:]=A[0:3,4*i+3].reshape(-1,1)-np.dot(R,B[0:3,4*i+3].reshape(-1,1))
    t,residuals,rank,s=np.linalg.lstsq(C,d,rcond=None)
    X1=np.hstack((R,t))
    X=np.vstack((X1,np.array([0,0,0,1])))
    return X

In [21]:
#主程序
def main():
    Pose1=[668.72,-160.34,223.71,-0.40,-0.08,64.97]
    fix_point1=[298,182.64,289,175.98,294,168.5,309,164.10]
    Pose2=[650.34,-197.27,207.11,1.63,-26.12,64.22]
    fix_point2=[312,178.20,293,173.25,286,164.70,317,154.80]
    Pose3=[693.75,-139.96,230.90,0.24,17.96,65.03]
    fix_point3=[309,187.92,287,177.30,294,171.58,313,167.62]
    #Pose4=[693.75,-138.96,230.90,0.24,17.96,65.03]
    #fix_point4=[309,187.92,287,154.30,294,171.58,313,167.62]
    #Pose4=list(map(float,input("请输入位姿4的数据:").split()))
    #将固定点的坐标从极坐标转化为直角坐标
    fix_point1=axis_transformtion(fix_point1)
    fix_point2=axis_transformtion(fix_point2)
    fix_point3=axis_transformtion(fix_point3)
    #fix_point4=axis_transformtion(fix_point4)
    #机械臂末端坐标系相比于机械臂末端坐标系的变换矩阵
    TB1=data_transform(Pose1)
    TB2=data_transform(Pose2)
    TB3=data_transform(Pose3)
    #TB4=data_transform(Pose4)
    R=246
    #标定板固定点相对于激光雷达的变化矩阵
    TC1=calculate_fixed_point(fix_point1,R,position=1)
    TC2=calculate_fixed_point(fix_point2,R,position=1)
    TC3=calculate_fixed_point(fix_point3,R,position=1)
    #TC4=calculate_fixed_point(fix_point3,R,position=1)
    #cir_point4=calculate_fixed_point(fix_point4,R,position=0)
    #参数说明:(fix_point:从激光雷达上读取的4个点的坐标;R:标定物球体的直径;position:如果激光扫到的圆面在大圆上方，则取值1，否则取值为0)
    TL1=np.dot(np.linalg.inv(TB1),TB2)
    TL2=np.dot(np.linalg.inv(TB2),TB3)
    #TL3=np.dot(np.linalg.inv(TB3),TB4)
    TR1=np.dot(TC1,np.linalg.inv(TC2))
    TR2=np.dot(TC2,np.linalg.inv(TC3))
    #TR3=np.dot(TC3,np.linalg.inv(TC4))
    A=np.hstack((TL1,TL2))
    B=np.hstack((TR1,TR2))
    X=tasi(A,B)
    print(X)
main()

C:\Users\youyiting\AppData\Local\Temp\ipykernel_17844\1587383431.py:15: RuntimeWarning: invalid value encountered in divide
  b=b/np.linalg.norm(b)


LinAlgError: SVD did not converge in Linear Least Squares